## Libraries

In [1]:
import numpy as np
import pandas as pd
import os
import torch
from torchvision import transforms, models
from torchvision.datasets import ImageFolder
from torch.utils.data import Dataset, DataLoader
from torch import nn
from PIL import Image
import pydicom
from sklearn.model_selection import train_test_split
import torchvision.models as modelst

## Config

In [2]:
class Config:
    BASE_DIR = '/kaggle/input/rsna-atd-512x512-png-v2-dataset'
    SEED = 12
    IMAGE_SIZE = (224, 224)
    BATCH_SIZE = 32
    TARGET_COLUMNS = ['bowel_healthy', 'bowel_injury',
                      'extravasation_healthy', 'extravasation_injury',
                      'kidney_healthy', 'kidney_low', 'kidney_high',
                      'liver_healthy', 'liver_low', 'liver_high',
                      'spleen_healthy', 'spleen_low', 'spleen_high',
                     ]
    
config = Config()

In [3]:
df = pd.read_csv(os.path.join(config.BASE_DIR, 'train.csv'))
df.head(5)

,patient_id,bowel_healthy,bowel_injury,extravasation_healthy,extravasation_injury,kidney_healthy,kidney_low,kidney_high,liver_healthy,liver_low,...,spleen_healthy,spleen_low,spleen_high,any_injury,series_id,instance_number,injury_name,image_path,width,height
0,10004,1,0,0,1,0,1,0,1,0,...,0,0,1,1,21057,362,Active_Extravasation,/kaggle/input/rsna-2023-abdominal-trauma-detec...,512,512
1,10004,1,0,0,1,0,1,0,1,0,...,0,0,1,1,21057,363,Active_Extravasation,/kaggle/input/rsna-2023-abdominal-trauma-detec...,512,512
2,10004,1,0,0,1,0,1,0,1,0,...,0,0,1,1,21057,364,Active_Extravasation,/kaggle/input/rsna-2023-abdominal-trauma-detec...,512,512
3,10004,1,0,0,1,0,1,0,1,0,...,0,0,1,1,21057,365,Active_Extravasation,/kaggle/input/rsna-2023-abdominal-trauma-detec...,512,512
4,10004,1,0,0,1,0,1,0,1,0,...,0,0,1,1,21057,366,Active_Extravasation,/kaggle/input/rsna-2023-abdominal-trauma-detec...,512,512


## train and validation set

In [4]:
def split_group(group, test_size=0.2):
    if len(group) == 1:
        return (group, pd.DataFrame()) if np.random.rand() < test_size else (pd.DataFrame(), group)
    else:
        return train_test_split(group, test_size=test_size, random_state=config.SEED)

train_set = pd.DataFrame()
validation_set = pd.DataFrame()

for _, group in df.groupby(config.TARGET_COLUMNS):
    train_group, val_group = split_group(group)
    train_set = pd.concat([train_set, train_group], ignore_index=True)
    validation_set = pd.concat([validation_set, val_group], ignore_index=True)
    
print(train_set.shape, validation_set.shape)

(10399, 21) (2616, 21)


In [5]:
train_set.sample(3)

,patient_id,bowel_healthy,bowel_injury,extravasation_healthy,extravasation_injury,kidney_healthy,kidney_low,kidney_high,liver_healthy,liver_low,...,spleen_healthy,spleen_low,spleen_high,any_injury,series_id,instance_number,injury_name,image_path,width,height
2319,57887,0,1,0,1,1,0,0,1,0,...,1,0,0,1,61745,230,Bowel,/kaggle/input/rsna-2023-abdominal-trauma-detec...,512,512
5479,61834,0,1,1,0,1,0,0,1,0,...,1,0,0,1,47652,133,Bowel,/kaggle/input/rsna-2023-abdominal-trauma-detec...,512,512
7694,4639,1,0,0,1,1,0,0,1,0,...,0,1,0,1,14738,116,Active_Extravasation,/kaggle/input/rsna-2023-abdominal-trauma-detec...,512,512


In [6]:
validation_set.sample(3)

,patient_id,bowel_healthy,bowel_injury,extravasation_healthy,extravasation_injury,kidney_healthy,kidney_low,kidney_high,liver_healthy,liver_low,...,spleen_healthy,spleen_low,spleen_high,any_injury,series_id,instance_number,injury_name,image_path,width,height
89,53908,0,1,0,1,1,0,0,0,1,...,1,0,0,1,15786,285,Bowel,/kaggle/input/rsna-2023-abdominal-trauma-detec...,512,512
2600,7411,1,0,0,1,1,0,0,1,0,...,1,0,0,1,42688,795,Active_Extravasation,/kaggle/input/rsna-2023-abdominal-trauma-detec...,512,512
318,56690,0,1,0,1,1,0,0,1,0,...,0,1,0,1,51951,347,Bowel,/kaggle/input/rsna-2023-abdominal-trauma-detec...,512,512


## Data pipeline

In [7]:
transformations = transforms.Compose([
        transforms.Resize(config.IMAGE_SIZE, interpolation=Image.NEAREST),
        transforms.ToTensor(),
        transforms.Lambda(lambda x : x / 255)
    ])

class Dataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transformations
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        
        image_path = self.df['image_path'][idx]
        dicom_file = pydicom.dcmread(image_path)
        pixel_array = dicom_file.pixel_array.astype(np.int16)
        image = Image.fromarray(pixel_array)
        image = self.transform(image)
        
        target = self.df[config.TARGET_COLUMNS].iloc[idx]
        target = torch.tensor(target.values, dtype=torch.float32)

        return image, target
        
dataset = Dataset(train_set, transformations)

In [8]:
dataloader = DataLoader(
    dataset,
    batch_size=config.BATCH_SIZE,
    shuffle=False
)

## Model

In [9]:
base = models.resnet152(pretrained=True)

# output for bowel and extravastaion
out_be = nn.Sequential(
    nn.Linear(base.fc.in_features, 512),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(512, 1),
    nn.Sigmoid() 
)

# output for kidney, liver and spleen
out_kls = nn.Sequential(
    nn.Linear(base.fc.in_features, 512),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(512, 3),
    nn.Softmax(dim=1)
)

class Model(nn.Module):
    def __init__(self, base, out_bowel, out_extravasation, out_kidney, out_liver, out_spleen):
        super(Model, self).__init__()
        self.base = base
        self.out_bowel = out_bowel
        self.out_extravasation = out_extravasation
        self.out_kidney = out_kidney
        self.out_liver = out_liver
        self.out_spleen = out_spleen
        
    def forward(self, x):
        x = self.base(x)
        out_bowel = self.out_bowel(x)
        out_extravasation = self.out_extravasation(x)
        out_kidney = self.out_kidney(x)
        out_liver = self.out_liver(x)
        out_spleen = self.out_spleen(x)
        
        return out_bowel, out_extravasation, out_kidney, out_liver, out_spleen 
    
model = Model(base, out_be, out_be, out_kls, out_kls, out_kls)

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet152_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet152_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet152-394f9c45.pth" to /root/.cache/torch/hub/checkpoints/resnet152-394f9c45.pth
100%|██████████| 230M/230M [00:03<00:00, 80.1MB/s]


In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

loss_be = nn.BCELoss()
loss_kls = nn.CrossEntropyLoss()

optimizer_bowel = torch.optim.Adam(out_be.parameters())
optimizer_extravasation = torch.optim.Adam(out_be.parameters())
optimizer_kidney = torch.optim.Adam(out_kls.parameters())
optimizer_liver = torch.optim.Adam(out_kls.parameters())
optimizer_spleen = torch.optim.Adam(out_kls.parameters())

## Training